In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

from itertools import chain

In [2]:
from scipy.stats import kstest
from scipy.stats import kruskal
from scipy.stats import mannwhitneyu
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from itertools import combinations
from sklearn.preprocessing import QuantileTransformer

In [3]:
from pprint import pprint

In [4]:
train_activity=pd.read_csv('../transformed/real_only_activity_int.csv')
test_activity=pd.read_csv('../transformed/test_real_datas_int.csv')

train_party=pd.read_csv('../lite_data/train_party_lite.csv')
test_party=pd.read_csv('../lite_data/test_party_lite.csv')

In [5]:
train_label=pd.read_csv('../lite_data/train_label_lite.csv')

In [6]:
#파티참여 유저와 그 유저가 참여한 주차 만들기
member_list=[[(p,t) for p in train_party.party_members_acc_id.iloc[i].split(',')] for i, t in enumerate(train_party.party_start_week)]

In [7]:
member_list[:2]

[[('ot160647', 1),
  ('ot47783', 1),
  ('tr53166', 1),
  ('ot200260', 1),
  ('ot148400', 1),
  ('tr97459', 1)],
 [('ot93483', 3),
  ('ot74426', 3),
  ('te4462', 3),
  ('tr75992', 3),
  ('tr89175', 3),
  ('tr89342', 3)]]

In [8]:
# 위의 튜플 자료형을 토대로 데이터 프레임만들기
total_member=pd.DataFrame(list(chain(*member_list)),columns=['new_id','wk'])
total_member.head(10)

,new_id,wk
0,ot160647,1
1,ot47783,1
2,tr53166,1
3,ot200260,1
4,ot148400,1
5,tr97459,1
6,ot93483,3
7,ot74426,3
8,te4462,3
9,tr75992,3


In [9]:
#그중에서 train user 명단만 따로 저장하기
train_member=total_member[total_member.new_id.str.contains('tr')]

In [10]:
# 중복된 경우 유저가 해당 주차에 참여한 횟수이므로 주차별로 집계해서 나타내기
train_member_df=train_member.groupby(['new_id','wk']).size().reset_index()

In [11]:
train_member_df.columns=['new_id','wk','party_cnt']

In [12]:
train_member_df.head(10)

,new_id,wk,party_cnt
0,tr1,8,3
1,tr100,1,16
2,tr100,2,8
3,tr100,3,1
4,tr1000,8,41
5,tr10000,8,44
6,tr10001,8,64
7,tr10002,1,20
8,tr10002,2,17
9,tr10002,3,4


In [13]:
# train_activity 셋과 서로 병합
merge_activity_party=pd.merge(train_activity,train_member_df,on=['new_id','wk'],how='outer')
merge_activity_party.isna().sum()

play_time                      660
npc_exp                        660
npc_hongmun                    660
quest_exp                      660
quest_hongmun                  660
item_hongmun                   660
game_combat_time               660
get_money                      660
duel_cnt                       660
duel_win                       660
partybattle_cnt                660
partybattle_win                660
cnt_enter_inzone_solo          660
cnt_enter_inzone_light         660
cnt_enter_inzone_skilled       660
cnt_enter_inzone_normal        660
cnt_enter_raid                 660
cnt_enter_raid_light           660
cnt_enter_bam                  660
cnt_clear_inzone_solo          660
cnt_clear_inzone_light         660
cnt_clear_inzone_skilled       660
cnt_clear_inzone_normal        660
cnt_clear_raid                 660
cnt_clear_raid_light           660
cnt_clear_bam                  660
normal_chat                    660
whisper_chat                   660
district_chat       

## 총 새로운 660개의 컬럼이 추가됨
> 새로 생성한 피쳐 - 'party_cnt' 의 경우 해당 주차의 각 유저의 파티 참여 횟수인데 이는 유저가 해당주차에 접속했어도 파티 활동을 안했으면 결측치가 당연히 생길수 있다

>하지만 train_activity 셋에서 각유저의 해당주차 활동이력이 없음에도 불구, 파티 참여 기록이 뜨는 660개의 관측치는 이상하다 생각하여 살펴보았다.

In [14]:
# 새롭게 만들어진 유저의 명단과 주차 탐색
merge_activity_party[merge_activity_party.cnt_dt.isna()].head(10)


,play_time,npc_exp,npc_hongmun,quest_exp,quest_hongmun,item_hongmun,game_combat_time,get_money,duel_cnt,duel_win,...,party_chat,guild_chat,faction_chat,cnt_use_buffitem,gathering_cnt,making_cnt,cnt_dt,wk,new_id,party_cnt
440323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,tr10080,1.0
440324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,tr10137,13.0
440325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,tr10357,1.0
440326,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,tr10413,8.0
440327,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,tr10995,18.0
440328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,tr11053,13.0
440329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,tr11079,1.0
440330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,tr11141,9.0
440331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,tr11152,1.0
440332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,tr1176,2.0


In [15]:
# 새로 생성된 관측치 목록
merge_activity_party.loc[merge_activity_party.cnt_dt.isna(),['new_id','wk','party_cnt']].head(10)

,new_id,wk,party_cnt
440323,tr10080,6,1.0
440324,tr10137,6,13.0
440325,tr10357,2,1.0
440326,tr10413,5,8.0
440327,tr10995,6,18.0
440328,tr11053,6,13.0
440329,tr11079,3,1.0
440330,tr11141,6,9.0
440331,tr11152,2,1.0
440332,tr1176,6,2.0


In [16]:
#알수없는 row에 대해 값만 따로 저장
detect_na=merge_activity_party.loc[merge_activity_party.cnt_dt.isna(),['new_id','wk']]

In [17]:
# 유저별 파티 시작 시간대 저장
start_time_list=[[(p,t) for p in train_party.party_members_acc_id.iloc[i].split(',')] for i, t in enumerate(train_party.party_start_time)]

In [18]:
# 튜플 데이터프레임으로 변환
total_start_list=pd.DataFrame(list(chain(*start_time_list)),columns=['new_id','party_start_time'])

In [19]:
#유저별 파티 종료 시간대 저장
end_time_list=[[(p,t) for p in train_party.party_members_acc_id.iloc[i].split(',')] for i, t in enumerate(train_party.party_end_time)]

In [20]:
# 튜플을 데이터 프레임으로 변환
total_end_list=pd.DataFrame(list(chain(*end_time_list)),columns=['new_id','party_end_time'])

In [21]:
# 유저의 파티 시작일 저장
day_list=[[(p,t) for p in train_party.party_members_acc_id.iloc[i].split(',')] for i, t in enumerate(train_party.party_start_day)]

In [22]:
# 튜플 데이터 프레임으로 변환
total_day_list=pd.DataFrame(list(chain(*day_list)),columns=['new_id','party_start_day'])

In [23]:
df_total=pd.concat([total_member,total_day_list,total_start_list,total_end_list],axis=1).iloc[:,[0,1,3,5,7]]

In [24]:
# train user 이력만 저장
df_train_party=df_total[df_total.new_id.str.contains('tr')]

In [25]:
df_train_party.head(20)

,new_id,wk,party_start_day,party_start_time,party_end_time
2,tr53166,1,1,09:14:58.558,09:41:30.200
5,tr97459,1,1,09:14:58.558,09:41:30.200
9,tr75992,3,3,11:05:05.176,13:07:42.515
10,tr89175,3,3,11:05:05.176,13:07:42.515
11,tr89342,3,3,11:05:05.176,13:07:42.515
14,tr16718,3,6,02:18:43.172,02:28:58.177
16,tr63569,3,6,02:18:43.172,02:28:58.177
19,tr15609,4,1,09:22:01.936,09:47:40.192
20,tr85158,4,1,09:22:01.936,09:47:40.192
21,tr36984,4,1,09:22:01.936,09:47:40.192


In [26]:
#위에 잘못 집계되엇다고 판단된 유저의 이력들과 위의 데이터 프레임 병합해보기
detect_na_user=pd.merge(detect_na,df_train_party,on=['new_id','wk'],how='left')
detect_na_user

,new_id,wk,party_start_day,party_start_time,party_end_time
0,tr10080,6,1,00:01:53.221,00:18:28.909
1,tr10137,6,1,00:32:03.408,00:48:38.635
2,tr10137,6,1,02:08:29.081,02:23:58.157
3,tr10137,6,1,01:02:44.382,01:16:14.784
4,tr10137,6,1,02:36:44.493,02:47:27.995
5,tr10137,6,1,00:12:02.723,00:22:38.322
6,tr10137,6,1,03:02:41.622,03:12:33.217
7,tr10137,6,1,02:24:59.981,02:35:07.863
8,tr10137,6,1,02:53:18.922,03:00:47.210
9,tr10137,6,1,00:50:27.538,01:00:45.339


In [27]:
detect_na_user.wk.value_counts()

6    3534
2     221
3     219
5     202
7     118
4      37
Name: wk, dtype: int64

In [28]:
detect_na_user.party_start_day.value_counts()

1    4331
Name: party_start_day, dtype: int64

In [29]:
detect_na_user.party_start_time.sort_values()

2369    00:00:06.698
1240    00:00:07.510
2971    00:00:15.493
3740    00:00:20.173
3969    00:00:22.872
1424    00:00:23.017
2300    00:00:23.017
1255    00:00:23.699
4314    00:00:23.699
1002    00:00:27.546
3224    00:00:31.158
1617    00:00:36.939
1070    00:00:38.639
479     00:00:41.032
3598    00:00:47.593
1196    00:01:17.290
3501    00:01:17.290
3460    00:01:17.290
2203    00:01:21.690
3478    00:01:26.199
2015    00:01:35.289
1509    00:01:48.291
0       00:01:53.221
3025    00:01:53.424
3995    00:01:58.401
2653    00:01:58.401
620     00:02:05.281
1619    00:02:10.304
549     00:02:10.585
2210    00:02:10.585
            ...     
351     04:59:03.712
1635    04:59:03.915
1626    04:59:05.440
3767    04:59:05.440
4297    04:59:11.945
3000    04:59:21.539
275     04:59:36.842
4165    04:59:36.842
1387    04:59:37.342
1355    04:59:47.019
1406    04:59:52.956
2938    04:59:52.956
984     04:59:55.437
703     04:59:55.437
1251    04:59:55.437
3747    04:59:56.139
544     04:59

In [30]:
detect_na_user.party_end_time.sort_values()

2971    00:00:18.972
3224    00:00:53.767
3740    00:01:07.883
4314    00:01:09.302
1255    00:01:09.302
3969    00:01:21.394
2203    00:01:54.687
1509    00:02:02.410
4302    00:02:10.694
3788    00:02:17.996
620     00:02:24.408
1196    00:02:32.583
3460    00:02:32.583
3501    00:02:32.583
1619    00:02:37.794
2204    00:02:44.113
875     00:02:44.612
3041    00:02:44.612
121     00:02:44.799
2666    00:02:44.799
549     00:02:56.391
2210    00:02:56.391
1901    00:03:14.116
4164    00:03:21.510
954     00:03:29.513
113     00:03:55.209
1002    00:03:57.055
2849    00:04:02.562
102     00:04:09.313
1871    00:04:22.012
            ...     
1406    05:03:13.950
2938    05:03:13.950
2927    05:03:19.052
2457    05:03:19.052
3453    05:03:19.052
1449    05:03:21.321
3457    05:03:21.321
3839    05:03:21.321
1626    05:03:22.554
3767    05:03:22.554
60      05:03:24.044
1979    05:03:24.044
1739    05:03:24.839
3069    05:03:24.839
2484    05:03:24.839
1646    05:03:30.440
1298    05:03


## 특징

### 잘못 집계된 유저들의 파티 활동 시간대는 새벽 00시~ 5시 5분이내,

### 시작 주는 다르지만 시작 일은 1로 동일하다


> 가정1. 각 주차 첫번째 날짜(party_start_day==1)에 새벽 00시~5시 5분까지 파티활동에 참여했던 유저들은 train_activity 셋에서 그 전주차 활동으로 집계 되었을 것이다.

> 해당 일자, 해당시간의 파티활동을 그 파티 생성 주의 전주(party_start_week-1)로 수정하고 진행 시도.

In [32]:
train_party.loc[ (train_party['party_start_day']==1) & (pd.to_timedelta(train_party['party_start_time'])<pd.Timedelta(6,'h')),
                'party_start_week']+=-1
    

In [34]:
transform_list=[[(p,t) for p in train_party.party_members_acc_id.iloc[i].split(',')] for i, t in enumerate(train_party.party_start_week)]

In [37]:
total_transform_list=pd.DataFrame(list(chain(*transform_list)),columns=['new_id','wk'])
train_list=total_transform_list[total_transform_list.new_id.str.contains('tr')]

In [39]:
train_list_ultimate=train_list.groupby(['new_id','wk']).size().reset_index()

In [42]:
train_list_ultimate.columns=['new_id','wk','party_cnt']

In [46]:
pd.merge(train_activity,train_list_ultimate,on=['new_id','wk'],how='outer').shape

(440323, 39)

In [47]:
train_activity.shape
# 수정해서 집계해보니 activity 데이터 셋 내 유저의 활동 이력 보존

(440323, 38)

##  결론

>  activity 변수에서 어떤 유저의 특정 주차에 활동 기록이 없음에도 불구하고 파티참여가 뜨는 이력이 있으므로 데이터 정제 단계에서 오류가 있었다고 판단했습니다.


> party 데이터셋 내에 해당조건 (각 주차 첫번째 날(paryt_start_day) 새벽00시~ 6시에 파티 참여활동)을 만족하는 경우, 유저 활동 주를 전주(party_start_week-1)로 수정하고 activity 데이터셋과 병합해보니 문제가 생기지 않았습니다.

> 앞으로 party 데이터와 activity 데이터를 함께 병합할 때 유저가 각 주차 첫번째 날(paryt_start_day) 새벽00시~ 6시에 파티 참여활동을 한 경우 전주로 집계하여 병합후 진행하기로 판단했습니다
